In [2]:
import numpy as np
import pandas as pd
import scipy.stats as st

# Data import


In [47]:

a = pd.read_csv("C:/Users/Tamir/Desktop/p01.txt",sep = '\t', nrows = 5, delimiter = ':',skipinitialspace = True, header = None, names = ['thing', 'nb'])
file_name = a.loc[0,'nb']
sample_size = int(a.loc[1,'nb'])
nb_attributes = int(a.loc[2,'nb'])
type_vector = a.loc[3,'nb']
type_vector = type_vector.split(' ')
nb_classes = int(a.loc[4,'nb'])
column_names = list()
for i in range(nb_attributes+1):
    column_names.append("attribute" + " " + str(i+1))
column_names[nb_attributes] = 'class'
data = pd.read_csv("C:/Users/Tamir/Desktop/p01.txt",sep = '\t', nrows = sample_size, delimiter = ' ',skiprows = 5, header = None,index_col = False, names = column_names)
max_depth = 4
root = [data]
level_1 = [None,None]
level_2 = [[None,None],[None,None]]
level_3 = [[None,None],[None,None],[None,None],[None,None]]
level_4 = [[None,None],[None,None],[None,None],[None,None],[None,None],[None,None],[None,None],[None,None]]
tree = (root,level_1,level_2,level_3,level_4) 
tree = list(tree)

In [198]:
def entropy(leaf):
    leaf_size = len(leaf)
    entropy = 0
    if leaf_size == 0:
        return(entropy,leaf_size)
    class_list = sorted(list(set(leaf['class'])))
    for i in class_list:
        class_size = len(data[data['class'] == i])
        relative_size = class_size/leaf_size
        entropy = entropy - relative_size*np.log(relative_size)
    return(entropy,leaf_size)
def infogain(parent,child_1,child_2):
    parent_entropy = entropy(parent)[0]
    child_1_info = entropy(child_1)
    child_2_info = entropy(child_2)
    child_1_size = child_1_info[1]
    child_2_size = child_2_info[1]
    child_1_entropy = child_1_info[0]
    child_2_entropy = child_2_info[0]
    infogain = 0
    if child_1_size == 0:
        child_1_size = 1
    if child_2_size ==0:
        child_2_size = 1
    infogain = parent_entropy - (child_1_entropy/child_1_size) - (child_2_entropy/child_2_size)
    return infogain
def split(leaf,attribute):
    column_name = "attribute" + " " + str(attribute+1)
    possible_values = sorted(list(set(leaf[column_name])))
    attribute_type = type_vector[attribute]
    child1_list = list()
    child2_list = list()
    IG_list = list()
    if attribute_type == 'C':
        #print('atributo é C')
        for i in possible_values:
            child_1 = leaf[leaf[column_name]== i]
            child_2 = leaf[leaf[column_name]!= i]
            child1_list.append(child_1)
            child2_list.append(child_2)
            IG_list.append(infogain(leaf,child_1,child_2))
    elif attribute_type == 'N':
        #print('atributo é N')
        #print('a lista de valores possíveis tem tamanho: ', len(possible_values))
        for i in possible_values:
            child_1 = leaf[leaf[column_name] >= i]
            child_2 = leaf[leaf[column_name] < i]
            child1_list.append(child_1)
            child2_list.append(child_2)
            IG_list.append(infogain(leaf,child_1,child_2))
    IG = max(IG_list)
    indice = IG_list.index(IG)
    child_1 = child1_list[indice]
    child_2 = child2_list[indice]
    return (IG,child_1,child_2,attribute)
def CART(tree,h,max_depth):
    #Quero criar uma lista de atributos pra entender a construção dessa árvore.
    #
    lista_atributo = list()
    while h < max_depth:
        x = 2**(h-1)
        if h == 0:
            x = 1
        a = 0
        print("altura atual é ", h)
        for j in range(x):
            if h == 0:
                complete_list = list()
                infogain_list = list()
                children_list = list()
                for attribute in range(nb_attributes):
                    complete_list.append(split(tree[h][j],attribute))
                    infogain_list.append(complete_list[attribute][0])
                indice = infogain_list.index(max(infogain_list))
                child_1 = complete_list[indice][1]
                child_2 = complete_list[indice][2]
                children_list.append((child_1,child_2))
                tree[1] = children_list
                melhor_atributo = complete_list[indice][3]
                lista_atributo.append(melhor_atributo)
            else:
                complete_list = list()
                infogain_list = list()
                children_list = list()
                for i in range(2):
                    for attribute in range(nb_attributes):
                        complete_list.append(split(tree[h][j][i],attribute)) 
                        infogain_list.append(complete_list[attribute][0])
                    indice = infogain_list.index(max(infogain_list))
                    child_1 = complete_list[indice][1]
                    child_2 = complete_list[indice][2]
                    children_list.append((child_1,child_2))
                    melhor_atributo = complete_list[indice][3]
                    lista_atributo.append(melhor_atributo)
                    tree[h+1][a] = children_list[0]
                    a = a + 1
        h = h + 1
    return(lista_atributo)
#O CART funciona, ele só gera nodes vazios. Ai a ideia é vc classificar msm

In [199]:
CART(tree,0,4)
#tree

altura atual é  0
altura atual é  1
altura atual é  2
altura atual é  3


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [191]:
tree[3][0][0]

,attribute 1,attribute 2,attribute 3,attribute 4,attribute 5,attribute 6,class
6,36.2,0,0,1,1,1,1
7,36.2,0,1,0,0,0,0
8,36.3,0,0,1,1,1,1
9,36.6,0,0,1,1,1,1
10,36.6,0,0,1,1,1,1
11,36.6,0,1,0,0,0,0
12,36.6,0,1,0,0,0,0
13,36.7,0,0,1,1,1,1
14,36.7,0,1,0,0,0,0
15,36.7,0,1,0,0,0,0


In [36]:
root = [data]
level_1 = [None,None]
level_2 = [[None,None],[None,None]]
level_3 = [[None,None],[None,None],[None,None],[None,None]]
level_4 = [[None,None],[None,None],[None,None],[None,None],[None,None],[None,None],[None,None],[None,None]]
tree = (root,level_1,level_2,level_3,level_4)        

In [188]:
tree[4][1][0]

,attribute 1,attribute 2,attribute 3,attribute 4,attribute 5,attribute 6,class
8,36.3,0,0,1,1,1,1
9,36.6,0,0,1,1,1,1
10,36.6,0,0,1,1,1,1
11,36.6,0,1,0,0,0,0
12,36.6,0,1,0,0,0,0
13,36.7,0,0,1,1,1,1
14,36.7,0,1,0,0,0,0
15,36.7,0,1,0,0,0,0
16,36.8,0,0,1,1,1,1
17,36.8,0,0,1,1,1,1


In [34]:
level_2 = [[None,None],[None,None]]
level_2

[[None, None], [None, None]]

In [128]:
tree[4][0][0]
tree[4][0][1]
tree[4][1][1]

,attribute 1,attribute 2,attribute 3,attribute 4,attribute 5,attribute 6,class


# Functions for entropy and infogain 

In [155]:

def best_split(node, attribute):
    IG_1 = -1
    column_name = "attribute" + " " + str(attribute+1)
    tipo = sorted(list(set(node[column_name])))
    if type_vector[attribute] == 'C':
        for j in tipo:
            child_1 = node[node[column_name] == j]
            child_2 = node[node[column_name] != j]
            if len(child_2) and len(child_1) >0:
                IG_2 = infogain(node,child_1,child_2)
                if  IG_2 > IG_1:
                    IG_1 = IG_2 
            else:
                next
    elif type_vector[attribute] == 'N':
        for k in tipo:
            child_1 = node[node[column_name] > k]
            child_2 = node[node[column_name] <=k]
            if len(child_1) and len(child_2) >0:
                #print('vim pra cá certin')
                IG_2 = infogain(node,child_1,child_2)
                if  IG_2 > IG_1:
                    IG_1 = IG_2 
            else:
                IG_2 = -
    return (IG_1,child_1,child_2)
#Essa função tá pegando um dataset, recebendo o atributo e fazendo o melhor split possível pra este atributo,
#Ela retorna o IG do split e o resultante do split. Talvez ele devesse retornar o parent?

In [206]:

tree = list()
tree.append(data)
h =0
for i in range(max_depth-1):
    tree.append(list())
node = tree[h]
#Beleza. Agora a árvore tem 4 listas. Uma de cada nível.
attribute = 4
IG_1 = -1
column_name = "attribute" + " " + str(attribute+1)
tipo = sorted(list(set(node[column_name])))



In [207]:
for k in tipo:
    child_1 = node[node[column_name] > k]
    child_2 = node[node[column_name] <=k]
    if len(child_1) and len(child_2) >0:
        IG_2 = infogain(node,child_1,child_2)
        if  IG_2 > IG_1:
            IG_1 = IG_2 
IG_1            

0

In [196]:
tree[h]

,attribute 1,attribute 2,attribute 3,attribute 4,attribute 5,attribute 6,class
0,35.5,0,1,0,0,0,0
1,35.9,0,0,1,1,1,1
2,35.9,0,1,0,0,0,0
3,36.0,0,0,1,1,1,1
4,36.0,0,1,0,0,0,0
5,36.0,0,1,0,0,0,0
6,36.2,0,0,1,1,1,1
7,36.2,0,1,0,0,0,0
8,36.3,0,0,1,1,1,1
9,36.6,0,0,1,1,1,1
